In [1]:
import requests as req
import json
import pandas as pd
import numpy as np
import networkx as nx
import re
import itertools
import time

In [3]:
ENTRY = "6qFt3TjvxMt77YGsktWG8Z"
CLIENT = "81e657278a2742739d2020bc48963bf5"
SECRET = "713df9a23eef4999aa75c2515b937b5f"
GENRE = '(rap|hip hop)'
LANG = '(franc|french|belg|franç)'

In [4]:
def queryToken():
    data = {
        'grant_type': 'client_credentials',
        'client_id': CLIENT,
        'client_secret': SECRET
    }
    res = req.post("https://accounts.spotify.com/api/token", data=data)
    t = json.loads(res.text)["access_token"]
    return t

def getNewArtists(graph):
    return [n for n in graph if "name" not in graph.nodes[n]]

def getUncheckedArtists(graph):
    return [n for n in [n for n in graph if "name" in graph.nodes[n]] if not graph.nodes[n]["checked"] and graph.nodes[n]["genre"]]

def queryArtists(graph, token, artists, limit=50):
    artists = [artists[i:i+limit] for i in range(0, len(artists), limit)]
    for _a in artists:
        headers = {'Authorization': 'Bearer '+token}
        url = "https://api.spotify.com/v1/artists?ids={}&limit={}".format(','.join(_a), limit)
        res = json.loads(req.get(url, headers=headers).text)
        for a in res["artists"]:
            G.add_node(a["id"], name=a["name"], genre=checkGenres(a["genres"]), checked=False, popularity=a["popularity"], id=a["id"])
        
def querySongs(graph, token, artist):
    headers = {'Authorization': 'Bearer '+token}
    url = "https://api.spotify.com/v1/artists/{}/albums".format(artist)
    res = json.loads(req.get(url, headers=headers).text)
    url = "https://api.spotify.com/v1/albums?ids={}".format(','.join([a["id"] for a in res["items"]]))
    res = json.loads(req.get(url, headers=headers).text)
    for album in res["albums"]:
        for track in album["tracks"]["items"]:
            for feat in itertools.combinations([artist["id"] for artist in track["artists"]], 2):
                if not graph.has_edge(*feat):
                    graph.add_edge(*feat, names="", ids="", count=0, datemax=0, datemin=10000)
                graph.edges[feat[0], feat[1]]["count"] += 1
                graph.edges[feat[0], feat[1]]["ids"] += ", "+track["id"]
                graph.edges[feat[0], feat[1]]["names"] += ", "+track["name"]
                graph.edges[feat[0], feat[1]]["datemin"] = min(int(album["release_date"][0:4]), graph.edges[feat[0], feat[1]]["datemin"])
                graph.edges[feat[0], feat[1]]["datemax"] = max(int(album["release_date"][0:4]), graph.edges[feat[0], feat[1]]["datemax"])
                
def checkGenres(genres):
    for g in genres:
        if re.search(GENRE, g) is not None:
            if re.search(LANG, g) is not None:
                return True
    return False

def step(graph, token, amount=1, sleep=0.01):
    for i in range(amount):
        print(i)
        for _artist in getUncheckedArtists(graph):
            querySongs(graph, token, _artist)
        time.sleep(0.01)
        queryArtists(graph, token, getNewArtists(G))

In [5]:
token = queryToken()

In [6]:
G = nx.Graph()
queryArtists(G, token, [ENTRY])

In [7]:
step(G, token, 6)

0
1
2
3
4
5


In [8]:
step(G, token, 1)

0


In [11]:
step(G, token, 1)

0


In [9]:
nx.write_graphml(G, "graph.graphml")

In [10]:
np.histogram(np.array([j for i,j in G.degree]))

(array([10230,   254,    68,    22,    15,    10,     3,     0,     1,
            1]),
 array([  1.,  29.,  57.,  85., 113., 141., 169., 197., 225., 253., 281.]))

In [2]:
G = nx.read_graphml("graph.graphml")

In [26]:
norm = {uv: k/733 for uv, k in nx.get_edge_attributes(G, "count").items()}
nx.set_edge_attributes(G, norm, "norm_count")

In [27]:
nx.write_graphml(G, "graph.graphml")